# Automated ML

In [18]:
import joblib

from azureml.core import Dataset, Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'sleep-health-project'

exp = Experiment(ws, experiment_name)

In [6]:
cluster_name = "sleep-health-compute"

# Verfiy that cluster does not exist already
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cluster.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
The Sleep Health and Lifestyle Dataset from Kaggle is used to perform a classification task. The data covers a wide range of variables related to sleep and daily habits. In the classification task it should be determined wether a person has a certain sleep disorder or none.

In [8]:
dataset = Dataset.get_by_name(ws, name='Sleep-Health-Dataset')
df = dataset.to_pandas_dataframe()

In [9]:
from train import clean_data


pandas.core.frame.DataFrame

In [10]:
df.columns

Index(['Person ID', 'Gender', 'Age', 'Occupation', 'Sleep Duration',
       'Quality of Sleep', 'Physical Activity Level', 'Stress Level',
       'BMI Category', 'Blood Pressure', 'Heart Rate', 'Daily Steps',
       'Sleep Disorder'],
      dtype='object')

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [13]:
automl_settings = {}

# Set parameters for AutoMLConfig
automl_config = AutoMLConfig(
    compute_target=cluster,
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=dataset,
    label_column_name="Sleep Disorder",
    n_cross_validations=5
)

In [15]:
# Submit automl run
remote_run = exp.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on sleep-health-compute with default configuration
Running on remote compute: sleep-health-compute


Experiment,Id,Type,Status,Details Page,Docs Page
sleep-health-project,AutoML_5e0da2bf-794e-4798-8cfa-aabcd0aaa516,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFea

## Run Details

In [19]:
RunDetails(remote_run).show()
remote_run.wait_for_completion()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_5e0da2bf-794e-4798-8cfa-aabcd0aaa516',
 'target': 'sleep-health-compute',
 'status': 'Completed',
 'startTimeUtc': '2023-07-03T11:30:48.153027Z',
 'endTimeUtc': '2023-07-03T12:02:02.603075Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'sleep-health-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"sleep-health-project","subscription_id":"cdbe0b43-92a0-4715-838a-f2648cc7ad21","resource_group":"aml-quickstarts-238132","workspace_name":"quick-starts-ws-238132","region":"southcentralus","compute_

## Best Model

In [20]:
best_run, model = remote_run.get_output()

Package:azureml-automl-runtime, training version:1.51.0.post2, current version:1.49.0
Package:azureml-core, training version:1.51.0, current version:1.49.0
Package:azureml-dataprep, training version:4.10.8, current version:4.9.1
Package:azureml-dataprep-rslex, training version:2.17.12, current version:2.16.1
Package:azureml-dataset-runtime, training version:1.51.0, current version:1.49.0
Package:azureml-defaults, training version:1.51.0, current version:1.49.0
Package:azureml-interpret, training version:1.51.0, current version:1.49.0
Package:azureml-mlflow, training version:1.51.0, current version:1.49.0
Package:azureml-pipeline-core, training version:1.51.0, current version:1.49.0
Package:azureml-responsibleai, training version:1.51.0, current version:1.49.0
Package:azureml-telemetry, training version:1.51.0, current version:1.49.0
Package:azureml-train-automl-client, training version:1.51.0.post1, current version:1.49.0
Package:azureml-train-automl-runtime, training version:1.51.0.po

In [21]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
sleep-health-project,AutoML_5e0da2bf-794e-4798-8cfa-aabcd0aaa516_31,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [22]:
model_path = "best_model_automl.pkl"
joblib.dump(model, model_path)
model = Model.register(model_path="path_to_model_file",model_name="my_model",workspace=workspace)

['best_model_automl.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
